In [135]:
import sys
import os
# import calib_eq_odds as eq
cwd = os.getcwd()
print(cwd)
sys.path.append(cwd + "/Post-Processing")
from calib_eq_odds import Model
import pandas as pd
import numpy as np
import torch
import torch.nn as nn


fn_rate = 0.5
fp_rate = 0.5

kiva_label = "dataAug/kiva_label.csv"
student_label = "dataAug/studentperf_label.csv" 
kiva_onehot = 'dataAug/kiva_onehot.csv'
student_onehot = 'dataAug/studentperf_onehot.csv'


/home/li3975/cs587


In [136]:
def read_kiva_label():
    
    data = pd.read_csv(kiva_label)
    data['repayment_interval'] = data['repayment_interval'].replace(2, 1)
    train_data = data [0:20000]
    target = train_data['repayment_interval']
    target = torch.tensor(target.values).float()
    X = train_data.drop(['repayment_interval'], axis=1)
    X = torch.tensor(X.values).float()
    gender_index = data.columns.get_loc('borrower_genders')
    val_data = data[20000: 40000]
    val_target = val_data['repayment_interval']
    val_target = torch.tensor(val_target.values).float()
    val_X = val_data.drop(['repayment_interval'], axis=1)
    val_X = torch.tensor(val_X.values).float()
    test_data = data[40000:]
    test_target = test_data['repayment_interval']
    test_target = torch.tensor(test_target.values).float()
    test_X = test_data.drop(['repayment_interval'], axis=1)
    test_X = torch.tensor(test_X.values).float()
    return X, gender_index, target, val_X, val_target, test_X, test_target
X, gender_index, target, val_X, val_target, test_X, test_target = read_kiva_label()

In [137]:
class LogisticRegression(nn.Module):
    def __init__(self, num_columns):
        # Call the parent constructor to initialize the nn.Module
        super(LogisticRegression, self).__init__()

        # Define a linear transformation layer (weights and bias) for the model.
        # `x_female.shape[1]` specifies the number of input features, and `out_features=1` indicates a single output for binary classification.
        # The bias term is included by setting bias=True, allowing the model to learn an intercept.
        self.w = nn.Linear(num_columns, out_features=1, bias=True)

        # Initialize a sigmoid activation function, which will be used to convert logits to probabilities.
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # Define the forward pass for the model.
        # Apply the linear transformation to the input `x`.
        w = self.w(x)

        # Apply the sigmoid function to the output of the linear layer.
        # This converts the logits (raw output of the linear layer) to probabilities in the range (0, 1).
        output = self.sigmoid(w)

        # Return the output probabilities.
        return output
    
def CVS(f_pred, m_pred, f_label, m_label, x_female, x_male):
    f_label = torch.tensor(f_label).float()
    m_label = torch.tensor(m_label).float()
    f_pred = torch.tensor(f_pred).float()
    f_pred = torch.where(f_pred > 0.5, 1, 0)
    m_pred = torch.tensor(m_pred).float()
    m_pred = torch.where(m_pred > 0.5, 1, 0)
    x_female = torch.stack(x_female)
    x_male = torch.stack(x_male)
    print()
    # Count the number of correct predictions for female samples.
    corr_f = torch.sum(f_pred == f_label)  # Count how many female predictions are True (1).
    # Count the number of correct predictions for male samples.
    corr_m = torch.sum(m_pred == m_label)  # Count how many male predictions are True (1).
    # Calculate the probability of predicting income as >50K for females.
    P_y1_s1 = corr_f / x_female.shape[0]  # Probability of positive predictions among females.
    # Calculate the probability of predicting income as >50K for males.
    P_y1_s0 = corr_m / x_male.shape[0]  # Probability of positive predictions among males.
    
    # Calculate the Calder-Verwer discrimination score, which measures the difference in positive prediction probabilities.
    CV_score = torch.abs(P_y1_s0 - P_y1_s1)  # Absolute difference between the two probabilities.
    
    # Return the Calder-Verwer discrimination score rounded to four decimal places.
    return round(CV_score.item(), 9)  


def DifferenceEqualOpportunity(f_pred, m_pred, f_label, m_label):
    f_label = torch.tensor(f_label).float()
    m_label = torch.tensor(m_label).float()
    f_pred = torch.tensor(f_pred).float()
    f_pred = torch.where(f_pred > 0.5, 1, 0)
    m_pred = torch.tensor(m_pred).float()
    m_pred = torch.where(m_pred > 0.5, 1, 0)
    TPrate_priv = torch.sum(m_pred * m_label) / torch.sum(m_label)
    TPrate_unpriv = torch.sum(f_pred * f_label) / torch.sum(f_label)
    TPrate_diff = torch.abs(TPrate_unpriv - TPrate_priv)
    return round(TPrate_diff.item(), 9)
def DifferenceAverageOdds(f_pred, m_pred, f_label, m_label):
    f_label = torch.tensor(f_label).float()
    m_label = torch.tensor(m_label).float()
    f_pred = torch.tensor(f_pred).float()
    f_pred = torch.where(f_pred > 0.5, 1, 0)
    m_pred = torch.tensor(m_pred).float()
    m_pred = torch.where(m_pred > 0.5, 1, 0)
    TPrate_priv = torch.sum(m_pred * m_label) / torch.sum(m_label)
    TPrate_unpriv = torch.sum(f_pred * f_label) / torch.sum(f_label)
    TPrate_diff = torch.abs(TPrate_unpriv - TPrate_priv)
    FPrate_priv = torch.sum(m_pred * (1 - m_label)) / torch.sum(1 - m_label)
    FPrate_unpriv = torch.sum(f_pred * (1 - f_label)) / torch.sum(1 - f_label)
    FPrate_diff = torch.abs(FPrate_unpriv - FPrate_priv)
    return round(((TPrate_diff + FPrate_diff) / 2).item(), 9)

In [138]:
def train(model):
    
    ceriation = nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    model.train()
    for epoch in range(100):
        optimizer.zero_grad()
        output = model(X)
        loss = ceriation(output.squeeze(1), target)
        loss.backward()
        optimizer.step()
        # print('epoch [{}/{}], loss:{:.4f}'.format(epoch + 1, 100, loss.item()))
    return model

In [139]:
model = LogisticRegression(9)
model = train(model)

In [140]:
# with no grad
group_0_val_pred = []
group_1_val_pred = []
group_0_val_gt= []
group_1_val_gt = []

group_0_test_pred = []
group_1_test_pred = []
group_0_test_gt= []
group_1_test_gt = []
group_0_test_x = []
group_1_test_x = []

def evaluated_nn(model, val_X, val_target, test_X, test_target, gender_index):

    model.eval()
    with torch.no_grad():
        for x, y in zip(val_X, val_target):
            y_pred = model(x)
            if x[gender_index] == 0:
                group_0_val_pred.append(y_pred.item())
                group_0_val_gt.append(y.item())
            else:
                group_1_val_pred.append(y_pred.item())
                group_1_val_gt.append(y.item())
            # print(f'Prediction: {y_pred.item()}, True: {y.item()}')
        
        for x, y in zip(test_X, test_target):
            y_pred = model(x)
            if x[gender_index] == 0:
                group_0_test_x.append(x)
                group_0_test_pred.append(y_pred.item())
                group_0_test_gt.append(y.item())
            else:
                group_1_test_x.append(x)
                group_1_test_pred.append(y_pred.item())
                group_1_test_gt.append(y.item())
            # print(f'Prediction: {y_pred.item()}, True: {y.item()}')

evaluated_nn(model, val_X, val_target, test_X, test_target, gender_index)

In [141]:
group_0_val_model = Model(np.array(group_0_val_pred), np.array(group_0_val_gt))
group_1_val_model = Model(np.array(group_1_val_pred), np.array(group_1_val_gt))
group_0_test_model = Model(np.array(group_0_test_pred), np.array(group_0_test_gt))
group_1_test_model = Model(np.array(group_1_test_pred), np.array(group_1_test_gt))
_, _, mix_rates = Model.calib_eq_odds(group_0_val_model, group_1_val_model, fp_rate, fn_rate)
calib_eq_odds_group_0_test_model, calib_eq_odds_group_1_test_model = Model.calib_eq_odds(
    group_0_test_model, group_1_test_model, fp_rate, fn_rate, mix_rates)

CV_score = CVS(calib_eq_odds_group_0_test_model.pred, calib_eq_odds_group_1_test_model.pred,
               calib_eq_odds_group_0_test_model.label, calib_eq_odds_group_1_test_model.label,
               group_0_test_x, group_1_test_x)
ACC = (calib_eq_odds_group_0_test_model.accuracy() + calib_eq_odds_group_1_test_model.accuracy()) / 2

DEO = DifferenceEqualOpportunity(calib_eq_odds_group_0_test_model.pred, calib_eq_odds_group_1_test_model.pred,
                                  calib_eq_odds_group_0_test_model.label, calib_eq_odds_group_1_test_model.label)
DAO = DifferenceAverageOdds(calib_eq_odds_group_0_test_model.pred, calib_eq_odds_group_1_test_model.pred,
                                   calib_eq_odds_group_0_test_model.label, calib_eq_odds_group_1_test_model.label)
print("Cv Score: ", CV_score)
print("DEO", DEO)
print("DAO", DAO)
print("F_DEO", ACC* (1-DEO))
print("F_DAO", ACC* (1-DAO))
# print('Original group 0 model:\n%s\n' % repr(group_0_test_model))
# print('Original group 1 model:\n%s\n' % repr(group_1_test_model))
# print('Equalized odds group 0 model:\n%s\n' % repr(calib_eq_odds_group_0_test_model))
# print('Equalized odds group 1 model:\n%s\n' % repr(calib_eq_odds_group_1_test_model))

# CVS_score = CVS(model, model, test_X[test_X[:, gender_index] == 0], test_X[test_X[:,gender_index] == 1])
# print("CVS score: ", CVS_score)


Cv Score:  0.142534554
DEO 0.064659953
DAO 0.206260026
F_DEO 0.8208401385414121
F_DAO 0.6965740773248608


In [142]:
def read_student_label():
    data = pd.read_csv(student_label)
    print(data.columns)
    average_score = data['Exam_Score'].mean()   
    data['Exam_Score'] = data['Exam_Score'].apply(lambda x: 1 if x > average_score else 0)
    train_data = data [0:2000]
    target = train_data['Exam_Score']
    target = torch.tensor(target.values).float()
    X = train_data.drop(['Exam_Score'], axis=1)
    X = torch.tensor(X.values).float()
    gender_index = data.columns.get_loc('Gender')
    val_data = data[2000: 4000]
    val_target = val_data['Exam_Score']
    val_target = torch.tensor(val_target.values).float()
    val_X = val_data.drop(['Exam_Score'], axis=1)
    val_X = torch.tensor(val_X.values).float()
    test_data = data[4000:]
    test_target = test_data['Exam_Score']
    test_target = torch.tensor(test_target.values).float()
    test_X = test_data.drop(['Exam_Score'], axis=1)
    test_X = torch.tensor(test_X.values).float()
    return X, gender_index, target, val_X, val_target, test_X, test_target

X, gender_index, target, val_X, val_target, test_X, test_target = read_student_label()

Index(['Unnamed: 0', 'Hours_Studied', 'Attendance', 'Sleep_Hours',
       'Previous_Scores', 'Motivation_Level', 'Tutoring_Sessions',
       'Family_Income', 'Teacher_Quality', 'Physical_Activity',
       'Learning_Disabilities', 'Parental_Education_Level',
       'Distance_from_Home', 'Gender', 'Exam_Score'],
      dtype='object')


In [143]:
model = LogisticRegression(14)
train(model)

group_0_val_pred = []
group_1_val_pred = []
group_0_val_gt= []
group_1_val_gt = []

group_0_test_pred = []
group_1_test_pred = []
group_0_test_gt= []
group_1_test_gt = []
group_0_test_x = []
group_1_test_x = []

evaluated_nn(model, val_X, val_target, test_X, test_target, gender_index)

In [144]:
group_0_val_model = Model(np.array(group_0_val_pred), np.array(group_0_val_gt))
group_1_val_model = Model(np.array(group_1_val_pred), np.array(group_1_val_gt))
group_0_test_model = Model(np.array(group_0_test_pred), np.array(group_0_test_gt))
group_1_test_model = Model(np.array(group_1_test_pred), np.array(group_1_test_gt))
_, _, mix_rates = Model.calib_eq_odds(group_0_val_model, group_1_val_model, fp_rate, fn_rate)
calib_eq_odds_group_0_test_model, calib_eq_odds_group_1_test_model = Model.calib_eq_odds(
    group_0_test_model, group_1_test_model, fp_rate, fn_rate, mix_rates)

CV_score = CVS(calib_eq_odds_group_0_test_model.pred, calib_eq_odds_group_1_test_model.pred,
               calib_eq_odds_group_0_test_model.label, calib_eq_odds_group_1_test_model.label,
               group_0_test_x, group_1_test_x)
ACC = (calib_eq_odds_group_0_test_model.accuracy() + calib_eq_odds_group_1_test_model.accuracy()) / 2

DEO = DifferenceEqualOpportunity(calib_eq_odds_group_0_test_model.pred, calib_eq_odds_group_1_test_model.pred,
                                  calib_eq_odds_group_0_test_model.label, calib_eq_odds_group_1_test_model.label)
DAO = DifferenceAverageOdds(calib_eq_odds_group_0_test_model.pred, calib_eq_odds_group_1_test_model.pred,
                                   calib_eq_odds_group_0_test_model.label, calib_eq_odds_group_1_test_model.label)
print("Cv Score: ", CV_score)
print("DEO", DEO)
print("DAO", DAO)
print("F_DEO", ACC* (1-DEO))
print("F_DAO", ACC* (1-DAO))


Cv Score:  0.007957995
DEO 0.070259526
DAO 0.046338998
F_DEO 0.5670964440813682
F_DAO 0.5816868020884659


# Dropping Sensitive Features

In [145]:
group_val_1 = []
group_val_0 = []
group_test_0 = []
group_test_1 = []


def read_kiva_label_drop():
    
    data = pd.read_csv(kiva_label)
    data['repayment_interval'] = data['repayment_interval'].replace(2, 1)
    train_data = data [0:20000]
    target = train_data['repayment_interval']
    target = torch.tensor(target.values).float()
    X = train_data.drop(['repayment_interval'], axis=1)
    X = X.drop(["borrower_genders"], axis=1)
    print(X.columns)
    X = torch.tensor(X.values).float()
    # gender_index = data.columns.get_loc('borrower_genders')
    val_data = data[20000: 40000]
    val_data1 = val_data[val_data['borrower_genders'] == 1]
    val_data0 = val_data[val_data['borrower_genders'] == 0]
    
    val_target1 = val_data1['repayment_interval']
    val_target1 = torch.tensor(val_target1.values).float()
    val_target0 = val_data0['repayment_interval']
    val_target0 = torch.tensor(val_target0.values).float()
    
    val_X1 = val_data1.drop(['repayment_interval'], axis=1)
    val_X1 = val_X1.drop(["borrower_genders"], axis=1)
    val_X1 = torch.tensor(val_X1.values).float()
    val_X0 = val_data0.drop(['repayment_interval'], axis=1)
    val_X0 = val_X0.drop(["borrower_genders"], axis=1)
    val_X0 = torch.tensor(val_X0.values).float()

    test_data = data[40000:]
    test_data1 = test_data[test_data['borrower_genders'] == 1]
    test_data0 = test_data[test_data['borrower_genders'] == 0]
    test_target1 = test_data1['repayment_interval']
    test_target1 = torch.tensor(test_target1.values).float()
    test_target0 = test_data0['repayment_interval']
    test_target0 = torch.tensor(test_target0.values).float()
    test_X1 = test_data1.drop(['repayment_interval'], axis=1)
    test_X1 = test_X1.drop(["borrower_genders"],  axis=1)
    test_X1 = torch.tensor(test_X1.values).float()
    test_X0 = test_data0.drop(['repayment_interval'], axis=1)
    test_X0 = test_X0.drop(["borrower_genders"], axis=1)
    test_X0 = torch.tensor(test_X0.values).float()

    # test_target = test_data['repayment_interval']
    # test_target = torch.tensor(test_target.values).float()
    # test_X = test_data.drop(['repayment_interval'], axis=1)
    # test_X = torch.tensor(test_X.values).float()
    return X, target, (val_X1, val_target1), (val_X0, val_target0), (test_X1, test_target1), (test_X0, test_target0)
X, target, val_1, val_0, test_1, test_0 = read_kiva_label_drop()


model = LogisticRegression(8)
train(model)



Index(['funded_amount', 'loan_amount', 'sector', 'country_code', 'currency',
       'partner_id', 'term_in_months', 'lender_count'],
      dtype='object')


LogisticRegression(
  (w): Linear(in_features=8, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

In [146]:

group_0_val_pred = []
group_1_val_pred = []
group_0_val_gt= []
group_1_val_gt = []

group_0_test_pred = []
group_1_test_pred = []
group_0_test_gt= []
group_1_test_gt = []
group_0_test_x = []
group_1_test_x = []

def evaluated_nn_drop(model, val_1, val_0, test_1, test_0):
    model.eval()
    with torch.no_grad():
        x,y = val_1
        for i in range(len(x)):
            y_pred = model(x[i])
            if(torch.isnan(y_pred)):
                print(x[i])
            group_1_val_pred.append(y_pred.item())
            group_1_val_gt.append(y[i].item())
        x,y = val_0
        for i in range(len(x)):
            y_pred = model(x[i])
            if(torch.isnan(y_pred)):
                print(x[i])
            group_0_val_pred.append(y_pred.item())
            group_0_val_gt.append(y[i].item())
        x,y = test_1
        for i in range(len(x)):
            group_1_test_x.append(x[i])
            y_pred = model(x[i])
            if(torch.isnan(y_pred)):
                print(x[i])
            group_1_test_pred.append(y_pred.item())
            group_1_test_gt.append(y[i].item())
        x,y = test_0
        for i in range(len(x)):
            group_0_test_x.append(x[i])
            y_pred = model(x[i])
            if(torch.isnan(y_pred)):
                print(x[i])
            group_0_test_pred.append(y_pred.item())
            group_0_test_gt.append(y[i].item())
evaluated_nn_drop(model, val_1, val_0, test_1, test_0)

In [147]:
print(group_0_val_pred)

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,

In [148]:
group_0_val_model = Model(np.array(group_0_val_pred), np.array(group_0_val_gt))
group_1_val_model = Model(np.array(group_1_val_pred), np.array(group_1_val_gt))
group_0_test_model = Model(np.array(group_0_test_pred), np.array(group_0_test_gt))
group_1_test_model = Model(np.array(group_1_test_pred), np.array(group_1_test_gt))
_, _, mix_rates = Model.calib_eq_odds(group_0_val_model, group_1_val_model, fp_rate, fn_rate)
calib_eq_odds_group_0_test_model, calib_eq_odds_group_1_test_model = Model.calib_eq_odds(
    group_0_test_model, group_1_test_model, fp_rate, fn_rate, mix_rates)

CV_score = CVS(calib_eq_odds_group_0_test_model.pred, calib_eq_odds_group_1_test_model.pred,
               calib_eq_odds_group_0_test_model.label, calib_eq_odds_group_1_test_model.label,
               group_0_test_x, group_1_test_x)
ACC = (calib_eq_odds_group_0_test_model.accuracy() + calib_eq_odds_group_1_test_model.accuracy()) / 2

DEO = DifferenceEqualOpportunity(calib_eq_odds_group_0_test_model.pred, calib_eq_odds_group_1_test_model.pred,
                                  calib_eq_odds_group_0_test_model.label, calib_eq_odds_group_1_test_model.label)
DAO = DifferenceAverageOdds(calib_eq_odds_group_0_test_model.pred, calib_eq_odds_group_1_test_model.pred,
                                   calib_eq_odds_group_0_test_model.label, calib_eq_odds_group_1_test_model.label)
print("Cv Score: ", CV_score)
print("DEO", DEO)
print("DAO", DAO)
print("F_DEO", ACC* (1-DEO))
print("F_DAO", ACC* (1-DAO))


Cv Score:  0.168472409
DEO 0.0
DAO 0.0
F_DEO 0.8646157888794033
F_DAO 0.8646157888794033


In [149]:
def read_student_label():
    data = pd.read_csv(student_label)
    print(data.columns)
    average_score = data['Exam_Score'].mean()   
    data['Exam_Score'] = data['Exam_Score'].apply(lambda x: 1 if x > average_score else 0)
    train_data = data [0:2000]
    target = train_data['Exam_Score']
    target = torch.tensor(target.values).float()
    X = train_data.drop(['Exam_Score'], axis=1)
    X = X.drop(['Gender'], axis=1)  
    X = torch.tensor(X.values).float()

    val_data = data[2000: 4000]
    val_data_1 = val_data[val_data['Gender'] == 1]
    val_data_0 = val_data[val_data['Gender'] == 0]
    
    val_target1 = val_data_1['Exam_Score']
    val_target1 = torch.tensor(val_target1.values).float()
    val_target0 = val_data_0['Exam_Score']
    val_target0 = torch.tensor(val_target0.values).float()
    
    val_X1 = val_data_1.drop(['Exam_Score'], axis=1)
    val_X1 = val_X1.drop(['Gender'], axis=1)
    val_X1 = torch.tensor(val_X1.values).float()
    val_X0 = val_data_0.drop(['Exam_Score'], axis=1)
    val_X0 = val_X0.drop(['Gender'], axis=1)  
    val_X0 = torch.tensor(val_X0.values).float()
        

    test_data = data[4000:]
    test_data0 = test_data[test_data['Gender'] == 0]
    test_data1 = test_data[test_data['Gender'] == 1]
    test_target0 = test_data0['Exam_Score']
    test_target0 = torch.tensor(test_target0.values).float()
    test_target1 = test_data1['Exam_Score']
    test_target1 = torch.tensor(test_target1.values).float()
    
    test_X0 = test_data0.drop(['Exam_Score'], axis=1)
    test_X0 = test_X0.drop(['Gender'], axis=1)
    test_X0 = torch.tensor(test_X0.values).float()
    test_X1 = test_data1.drop(['Exam_Score'], axis=1)
    test_X1 = test_X1.drop(['Gender'], axis=1)
    test_X1 = torch.tensor(test_X1.values).float()
    
    return X, target, (val_X1, val_target1), (val_X0, val_target0), (test_X1, test_target1), (test_X0, test_target0)
X, target, val_1, val_0, test_1, test_0 = read_kiva_label_drop()

Index(['funded_amount', 'loan_amount', 'sector', 'country_code', 'currency',
       'partner_id', 'term_in_months', 'lender_count'],
      dtype='object')


In [150]:
model = LogisticRegression(8)
train(model)

group_0_val_pred = []
group_1_val_pred = []
group_0_val_gt= []
group_1_val_gt = []

group_0_test_pred = []
group_1_test_pred = []
group_0_test_gt= []
group_1_test_gt = []
group_0_test_x = []
group_1_test_x = []
evaluated_nn_drop(model, val_1, val_0, test_1, test_0)

In [151]:
group_0_val_model = Model(np.array(group_0_val_pred), np.array(group_0_val_gt))
group_1_val_model = Model(np.array(group_1_val_pred), np.array(group_1_val_gt))
group_0_test_model = Model(np.array(group_0_test_pred), np.array(group_0_test_gt))
group_1_test_model = Model(np.array(group_1_test_pred), np.array(group_1_test_gt))
_, _, mix_rates = Model.calib_eq_odds(group_0_val_model, group_1_val_model, fp_rate, fn_rate)
calib_eq_odds_group_0_test_model, calib_eq_odds_group_1_test_model = Model.calib_eq_odds(
    group_0_test_model, group_1_test_model, fp_rate, fn_rate, mix_rates)

CV_score = CVS(calib_eq_odds_group_0_test_model.pred, calib_eq_odds_group_1_test_model.pred,
               calib_eq_odds_group_0_test_model.label, calib_eq_odds_group_1_test_model.label,
               group_0_test_x, group_1_test_x)
ACC = (calib_eq_odds_group_0_test_model.accuracy() + calib_eq_odds_group_1_test_model.accuracy()) / 2

DEO = DifferenceEqualOpportunity(calib_eq_odds_group_0_test_model.pred, calib_eq_odds_group_1_test_model.pred,
                                  calib_eq_odds_group_0_test_model.label, calib_eq_odds_group_1_test_model.label)
DAO = DifferenceAverageOdds(calib_eq_odds_group_0_test_model.pred, calib_eq_odds_group_1_test_model.pred,
                                   calib_eq_odds_group_0_test_model.label, calib_eq_odds_group_1_test_model.label)
print("Cv Score: ", CV_score)
print("DEO", DEO)
print("DAO", DAO)
print("F_DEO", ACC* (1-DEO))
print("F_DAO", ACC* (1-DAO))


Cv Score:  0.146079719
DEO 0.03280735
DAO 0.125671506
F_DEO 0.8470790651180081
F_DAO 0.76574750987154


# Onehot Encoding

In [152]:
def read_kiva_onehot():
    
    data = pd.read_csv(kiva_onehot)    
    print(data.columns)
    data['repayment_interval'] = data['repayment_interval'].replace(2, 1)
    train_data = data [0:20000]
    target = train_data['repayment_interval']
    target = torch.tensor(target.values).float()
    X = train_data.drop(['repayment_interval'], axis=1)
    X = torch.tensor(X.values).float()
    gender_index_male = data.columns.get_loc('borrower_genders_male')
    gender_index_female = data.columns.get_loc('borrower_genders_female')
    val_data = data[20000: 40000]
    val_target = val_data['repayment_interval']
    val_target = torch.tensor(val_target.values).float()
    val_X = val_data.drop(['repayment_interval'], axis=1)
    val_X = torch.tensor(val_X.values).float()
    test_data = data[40000:]
    test_target = test_data['repayment_interval']
    test_target = torch.tensor(test_target.values).float()
    test_X = test_data.drop(['repayment_interval'], axis=1)
    test_X = torch.tensor(test_X.values).float()
    return X, (gender_index_male, gender_index_female), target, val_X, val_target, test_X, test_target
X, gender_index, target, val_X, val_target, test_X, test_target = read_kiva_onehot()

Index(['funded_amount', 'loan_amount', 'partner_id', 'term_in_months',
       'lender_count', 'repayment_interval', 'country_code_AF',
       'country_code_AL', 'country_code_AM', 'country_code_AZ',
       ...
       'sector_Health', 'sector_Housing', 'sector_Manufacturing',
       'sector_Personal Use', 'sector_Retail', 'sector_Services',
       'sector_Transportation', 'sector_Wholesale', 'borrower_genders_female',
       'borrower_genders_male'],
      dtype='object', length=163)


In [153]:
model = LogisticRegression(162)
train(model)

LogisticRegression(
  (w): Linear(in_features=162, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

In [154]:
group_0_val_pred = []
group_1_val_pred = []
group_0_val_gt= []
group_1_val_gt = []

group_0_test_pred = []
group_1_test_pred = []
group_0_test_gt= []
group_1_test_gt = []
group_0_test_x = []
group_1_test_x = []
def evaluated_nn_onehot(model, val_X, val_target, test_X, test_target, gender_index):
    gi_male, gi_female = gender_index
    model.eval()
    with torch.no_grad():
        for x, y in zip(val_X, val_target):
            y_pred = model(x)
            if x[gi_female] == 0:
                group_0_val_pred.append(y_pred.item())
                group_0_val_gt.append(y.item())
            else:
                group_1_val_pred.append(y_pred.item())
                group_1_val_gt.append(y.item())
            # print(f'Prediction: {y_pred.item()}, True: {y.item()}')
        
        for x, y in zip(test_X, test_target):
            y_pred = model(x)
            if x[gi_female] == 0:
                group_0_test_x.append(x)
                group_0_test_pred.append(y_pred.item())
                group_0_test_gt.append(y.item())
            else:
                group_1_test_x.append(x)
                group_1_test_pred.append(y_pred.item())
                group_1_test_gt.append(y.item())
            # print(f'Prediction: {y_pred.item()}, True: {y.item()}')
evaluated_nn_onehot(model, val_X, val_target, test_X, test_target, gender_index)

In [155]:
group_0_val_model = Model(np.array(group_0_val_pred), np.array(group_0_val_gt))
group_1_val_model = Model(np.array(group_1_val_pred), np.array(group_1_val_gt))
group_0_test_model = Model(np.array(group_0_test_pred), np.array(group_0_test_gt))
group_1_test_model = Model(np.array(group_1_test_pred), np.array(group_1_test_gt))
_, _, mix_rates = Model.calib_eq_odds(group_0_val_model, group_1_val_model, fp_rate, fn_rate)
calib_eq_odds_group_0_test_model, calib_eq_odds_group_1_test_model = Model.calib_eq_odds(
    group_0_test_model, group_1_test_model, fp_rate, fn_rate, mix_rates)

CV_score = CVS(calib_eq_odds_group_0_test_model.pred, calib_eq_odds_group_1_test_model.pred,
               calib_eq_odds_group_0_test_model.label, calib_eq_odds_group_1_test_model.label,
               group_0_test_x, group_1_test_x)
ACC = (calib_eq_odds_group_0_test_model.accuracy() + calib_eq_odds_group_1_test_model.accuracy()) / 2

DEO = DifferenceEqualOpportunity(calib_eq_odds_group_0_test_model.pred, calib_eq_odds_group_1_test_model.pred,
                                  calib_eq_odds_group_0_test_model.label, calib_eq_odds_group_1_test_model.label)
DAO = DifferenceAverageOdds(calib_eq_odds_group_0_test_model.pred, calib_eq_odds_group_1_test_model.pred,
                                   calib_eq_odds_group_0_test_model.label, calib_eq_odds_group_1_test_model.label)
print("Cv Score: ", CV_score)
print("DEO", DEO)
print("DAO", DAO)
print("F_DEO", ACC* (1-DEO))
print("F_DAO", ACC* (1-DAO))


Cv Score:  0.158131123
DEO 0.021263063
DAO 0.035906136
F_DEO 0.8533029475955627
F_DAO 0.8405365168209602


In [156]:
def read_student_onehot():
    data = pd.read_csv(student_onehot)
    print(data.columns)
    average_score = data['Exam_Score'].mean()   
    data['Exam_Score'] = data['Exam_Score'].apply(lambda x: 1 if x > average_score else 0)
    train_data = data [0:2000]
    target = train_data['Exam_Score']
    target = torch.tensor(target.values).float()
    X = train_data.drop(['Exam_Score'], axis=1)
    X = torch.tensor(X.values).float()
    gender_index_male = data.columns.get_loc('Gender_Male')
    gender_index_female = data.columns.get_loc('Gender_Female')
    val_data = data[2000: 4000]
    val_target = val_data['Exam_Score']
    val_target = torch.tensor(val_target.values).float()
    val_X = val_data.drop(['Exam_Score'], axis=1)
    val_X = torch.tensor(val_X.values).float()
    test_data = data[4000:]
    test_target = test_data['Exam_Score']
    test_target = torch.tensor(test_target.values).float()
    test_X = test_data.drop(['Exam_Score'], axis=1)
    test_X = torch.tensor(test_X.values).float()
    return X, (gender_index_male, gender_index_female), target, val_X, val_target, test_X, test_target

X, gender_index, target, val_X, val_target, test_X, test_target = read_student_onehot()

Index(['Unnamed: 0', 'Hours_Studied', 'Attendance', 'Sleep_Hours',
       'Previous_Scores', 'Tutoring_Sessions', 'Physical_Activity',
       'Exam_Score', 'Motivation_Level_High', 'Motivation_Level_Low',
       'Motivation_Level_Medium', 'Family_Income_High', 'Family_Income_Low',
       'Family_Income_Medium', 'Teacher_Quality_High', 'Teacher_Quality_Low',
       'Teacher_Quality_Medium', 'Learning_Disabilities_No',
       'Learning_Disabilities_Yes', 'Parental_Education_Level_College',
       'Parental_Education_Level_High School',
       'Parental_Education_Level_Postgraduate', 'Distance_from_Home_Far',
       'Distance_from_Home_Moderate', 'Distance_from_Home_Near',
       'Gender_Female', 'Gender_Male'],
      dtype='object')


In [157]:
model = LogisticRegression(26)
train(model)
group_0_val_pred = []
group_1_val_pred = []
group_0_val_gt= []
group_1_val_gt = []

group_0_test_pred = []
group_1_test_pred = []
group_0_test_gt= []
group_1_test_gt = []
group_0_test_x = []
group_1_test_x = []

evaluated_nn_onehot(model, val_X, val_target, test_X, test_target, gender_index)

In [158]:
group_0_val_model = Model(np.array(group_0_val_pred), np.array(group_0_val_gt))
group_1_val_model = Model(np.array(group_1_val_pred), np.array(group_1_val_gt))
group_0_test_model = Model(np.array(group_0_test_pred), np.array(group_0_test_gt))
group_1_test_model = Model(np.array(group_1_test_pred), np.array(group_1_test_gt))
_, _, mix_rates = Model.calib_eq_odds(group_0_val_model, group_1_val_model, fp_rate, fn_rate)
calib_eq_odds_group_0_test_model, calib_eq_odds_group_1_test_model = Model.calib_eq_odds(
    group_0_test_model, group_1_test_model, fp_rate, fn_rate, mix_rates)

CV_score = CVS(calib_eq_odds_group_0_test_model.pred, calib_eq_odds_group_1_test_model.pred,
               calib_eq_odds_group_0_test_model.label, calib_eq_odds_group_1_test_model.label,
               group_0_test_x, group_1_test_x)
ACC = (calib_eq_odds_group_0_test_model.accuracy() + calib_eq_odds_group_1_test_model.accuracy()) / 2

DEO = DifferenceEqualOpportunity(calib_eq_odds_group_0_test_model.pred, calib_eq_odds_group_1_test_model.pred,
                                  calib_eq_odds_group_0_test_model.label, calib_eq_odds_group_1_test_model.label)
DAO = DifferenceAverageOdds(calib_eq_odds_group_0_test_model.pred, calib_eq_odds_group_1_test_model.pred,
                                   calib_eq_odds_group_0_test_model.label, calib_eq_odds_group_1_test_model.label)
print("Cv Score: ", CV_score)
print("DEO", DEO)
print("DAO", DAO)
print("F_DEO", ACC* (1-DEO))
print("F_DAO", ACC* (1-DAO))


Cv Score:  0.004320741
DEO 0.006064855
DAO 0.008281771
F_DEO 0.5965303606311704
F_DAO 0.5951998334759313
